In [148]:
import gspread_dataframe as gd

In [149]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [150]:
#opening the google spreadsheet
ws = gc.open("Copy of Fetch Amazon Prices").sheet1

In [151]:
#google spreadsheet to dataframe
existing = gd.get_as_dataframe(ws)

In [152]:
import requests
from bs4 import BeautifulSoup
import datetime as dt
list4 = []
for i in  range(0,10):
    
    list3 = []
    URL = existing['Product URL'][i]

    headers = {"User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}


    page = requests.get(URL,headers=headers)

    soup = BeautifulSoup(page.content,'html.parser')

    #checking using soup.find(),which id is present for particular product as for the given data there are 4types of id : #priceblock_dealprice,#priceblock_ourprice,#priceblock_saleprice and #mbc-price-1
    if(soup.find(id="priceblock_dealprice")):
      price = soup.find(id="priceblock_dealprice").get_text()
      
    elif(soup.find(id="priceblock_ourprice")):
      price = soup.find(id="priceblock_ourprice").get_text()
      
    elif(soup.find(id="mbc-price-1")):
      price = soup.find(id="mbc-price-1").get_text()
      price = str(price)[1:-1]
      
    else:
      price = soup.find(id="priceblock_saleprice").get_text()
      
    
    existing['Price of Product'][i] = price
    #date of fetching
    existing['Date of Fetching'][i] = dt.datetime.today().strftime("%m/%d/%Y")


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [153]:
existing.drop(existing.columns[existing.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [154]:
existing

,Date of Fetching,Product URL,Price of Product
0,07/24/2020,https://www.amazon.in/Apple-iPhone-11-128GB-Wh...,"₹ 69,600.00"
1,07/24/2020,https://www.amazon.in/Grand-Theft-V-Premium-PS...,"1,599.00"
2,07/24/2020,https://www.amazon.in/Russell-Hobbs-RIC1400X-I...,"₹ 2,399.00"
3,07/24/2020,https://www.amazon.in/dp/B07NJRTCN9,"₹ 4,699.00"
4,07/24/2020,https://www.amazon.in/dp/B07W6T8H5P,"₹ 10,499.00"
...,...,...,...
994,NaN,NaN,NaN
995,NaN,NaN,NaN
996,NaN,NaN,NaN
997,NaN,NaN,NaN


In [155]:
#appending google sheet with the new values
gd.set_with_dataframe(ws,existing)